In [1]:
import os

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

e:\langgraph_end_to_end\genai_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = 'gemini-1.0-pro')

In [7]:
llm.invoke("Hi,I am shoaib!1")

AIMessage(content="Hi Shoaib, it's nice to meet you!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-ff92cf2f-8ffa-402a-a0ba-3822feae0889-0', usage_metadata={'input_tokens': 9, 'output_tokens': 12, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}})

### Wikipedia search tool

In [8]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [9]:
api_wrapper = WikipediaAPIWrapper(top_k_results = 5, doc_content_char_max = 100)

In [11]:
tool = WikipediaQueryRun(api_wrapper = api_wrapper)

In [12]:
tool.name

'wikipedia'

In [13]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [14]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [18]:
print(tool.run({"query":"What is langchain"}))

Page: Retrieval-augmented generation
Summary: Retrieval Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  
Use cases include providing chatbot access to internal company data or giving factual information only from an authoritative source.



Page: Prompt injection
Summary: Prompt injection is a family of related computer security exploits carried out by getting a machine learning model (such as an LLM) which was trained to follow human-given instructions to follow instructions provided by a malicious user. This stands in contrast to the intended operation of instruction-following systems, wherein t

## Create Custom tool

In [19]:
from langchain.agents import tool
@tool
def get_word_length(word:str)->int:
    '''Returns the length of a word'''
    return len(word)

In [20]:
get_word_length.invoke("Shoaib")

6

In [21]:
@tool 
def multiply(a:int,b:int)->int:
    '''Multiply two numbers'''
    return a*b

In [22]:
multiply.invoke({"a":10,"b":20})

200

In [23]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


## Concept of AGENTS


In [24]:
# This AGENT class from the first verion of langchain
from langchain.agents import AgentType, load_tools, initialize_agent

In [28]:
tool = load_tools(["wikipedia"],llm = llm)

In [32]:
agent = initialize_agent(tool,llm,agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose = True,handle_parsing_errors=True)

In [33]:
agent.run("What is langchain and who created llm model")



> Entering new AgentExecutor chain...
Action: wikipedia
Action Input: Langchain
Observation: Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Milvus (vector database)
Summary: Milvus is a distributed vector database developed by Zilliz. It is available as both open-source software and a cloud service.
Milvus is an open-source project under LF AI & Data Foundation distributed under the Apache License 2.0.

Page: Retrieval-augmented generation
Summary: Retrieval Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user qu

'I am unable to answer the question based on the information provided.'

In [34]:
agent.run("What is the gdp of india country?")



> Entering new AgentExecutor chain...
Action: wikipedia
Action Input: What is the GDP of India?
Observation: Page: List of countries by GDP (nominal)
Summary: Gross domestic product (GDP) is the market value of all final goods and services from a nation in a given year. Countries are sorted by nominal GDP estimates from financial and statistical institutions, which are calculated at market or government official exchange rates. Nominal GDP does not take into account differences in the cost of living in different countries, and the results can vary greatly from one year to another based on fluctuations in the exchange rates of the country's currency. Such fluctuations may change a country's ranking from one year to the next, even though they often make little or no difference in the standard of living of its population.
Comparisons of national wealth are also frequently made based on purchasing power parity (PPP), to adjust for differences in the cost of living in different countries.

'3.46 trillion USD'

In [47]:
# Second Tool 
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-pro')
from langchain_community.tools.tavily_search import TavilySearchResults

In [48]:
search = TavilySearchResults()

In [49]:
search.invoke("What is the weather in Lucknow?")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Lucknow', 'region': 'Uttar Pradesh', 'country': 'India', 'lat': 26.85, 'lon': 80.917, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1733250133, 'localtime': '2024-12-03 23:52'}, 'current': {'last_updated_epoch': 1733249700, 'last_updated': '2024-12-03 23:45', 'temp_c': 18.8, 'temp_f': 65.8, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 6.7, 'wind_kph': 10.8, 'wind_degree': 283, 'wind_dir': 'WNW', 'pressure_mb': 1014.0, 'pressure_in': 29.96, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 38, 'cloud': 0, 'feelslike_c': 18.8, 'feelslike_f': 65.8, 'windchill_c': 18.8, 'windchill_f': 65.8, 'heatindex_c': 18.8, 'heatindex_f': 65.8, 'dewpoint_c': 4.2, 'dewpoint_f': 39.6, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 14.1, 'gust_kph': 22.7}}"},
 {'url': 'https://www.livehindustan.com/weather/lucknow-me-aaj-ka-mausam-3-decemb

In [50]:
tools = [search]

In [51]:
from langchain import hub

In [52]:
prompt = hub.pull("hwchase17/openai-functions-agent")

e:\langgraph_end_to_end\genai_env\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [53]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [54]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm,tools,prompt)

In [55]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)

In [56]:
agent_executor.invoke({"input":"Hello how are you?"})



> Entering new AgentExecutor chain...
I am doing well, how can I help you today?


> Finished chain.


{'input': 'Hello how are you?',
 'output': 'I am doing well, how can I help you today?\n'}

In [57]:
agent_executor.invoke({"input":"What is the weather in Lucknow?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in Lucknow'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Lucknow', 'region': 'Uttar Pradesh', 'country': 'India', 'lat': 26.85, 'lon': 80.917, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1733250133, 'localtime': '2024-12-03 23:52'}, 'current': {'last_updated_epoch': 1733249700, 'last_updated': '2024-12-03 23:45', 'temp_c': 18.8, 'temp_f': 65.8, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 6.7, 'wind_kph': 10.8, 'wind_degree': 283, 'wind_dir': 'WNW', 'pressure_mb': 1014.0, 'pressure_in': 29.96, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 38, 'cloud': 0, 'feelslike_c': 18.8, 'feelslike_f': 65.8, 'windchill_c': 18.8, 'windchill_f': 65.8, 'heatindex_c': 18.8, 'heatindex_f': 65.8, 'dewpoint_c': 4.2, 'dewpoint_f': 39.6, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'g

{'input': 'What is the weather in Lucknow?',
 'output': 'The weather in Lucknow, India is currently clear with a temperature of 18.8°C. The wind is blowing from the west-northwest at 10.8 km/h. The humidity is 38%.\n'}